# **재무제표 데이터 수집**

<br>
## **1. Nice 정보통신 Site를 통해서 재무제표 수집**
Pandas를 활용하여 데이터 수집

In [1]:
import pandas as pd
url = "http://media.kisline.com/highlight/mainHighlight.nice?nav=1&paper_stock=000660"
df = pd.read_html(url)
# for no in range(len(df)):
#     print('Data Number is', no,'\n\n', df[no].head(3), '\n\n\n\n')

df_fin         = pd.concat([ df[6], df[7]], axis=1)
df_fin.columns = [i[1]  for i in df_fin.columns]
cols_fin       = df_fin["구분"].tolist()
del df_fin["구분"]
df_fin         = df_fin.T.sort_index().T
df_fin.index   = cols_fin
df_fin         = df_fin.T
df_fin

,매출액(억원),영업이익(억원),영업이익율(%),당기순이익(억원),순이익율(%),자산총계(억원),부채총계(억원),자본총계(억원),유보율(%),ROE,PER,PBR,PSR,EPS(원),BPS(원),SPS(원),EBITDA(억원),EV/EBITDA
2015.12,187980,53361,28.39,43224,22.99,296779,82902,213877,505.82,21.93,5.12,1.05,1.18,6002,29377,26103,-,-
2016.12,171980,32767,19.05,29538,17.18,322160,81925,240235,577.73,13.01,10.68,1.35,1.83,4184,32990,24360,-,-
2016.12,53577,15361,28.67,16267,0,322160,81925,240235,0,0,0,1.35,0,0,32990,0,-,-
2017.03,62895,24676,39.23,18980,30.19,343195,91506,251689,609.04,30.87,18.79,1.46,5.67,2688,34564,8909,-,-
2017.06,66923,30507,45.59,24685,36.89,375016,97378,277638,679.98,37.32,19.27,1.77,7.11,3497,38128,9479,-,-
2017.09,81001,37372,46.14,30542,37.72,407305,98260,309045,765.82,41.66,19.16,1.95,7.23,4326,42440,11473,-,-
2017.12,301094,137213,45.57,106415,35.34,454185,115975,338209,845.61,36.8,5.08,1.65,1.79,15073,46449,42648,-,-
2017.12,90276,44658,49.47,32207,0,454185,115975,338209,0,0,0,1.65,0,0,46449,0,-,-
2018.12(E),375256,176033,46.91,136390,36.35,-,-,-,-,33.62,4.13,1.2,-,18824,64614,-,-,-


<br>
## **2. Dart API를 활용한 기업정보 수집**
https://datascienceschool.net/view-notebook/adead36729704e7b8660dda3be6a6524/
1. 금융감독원 인증키(Dart.kss.go.kr)
1. 인증키 : 8e30368d048361ba865f916c30739160a0e58ddd
1. 발급일 : 2017-08-03
1. 제한 횟수 : 10,000
1. 가능한 기능
    1. 기업 개황 제공
    1. 공시 정보 검색

<br>
### **1) 기업개황 데이터 수집**
응답결과는 JSON 파일 형식으로 각 필드의 의미는 다음과 같다.
1. acc_mt : 결산월(MM)
1. bsn_no : 사업자등록번호
1. adr : 주소
1. crp_cls : 법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
1. crp_no : 법인등록번호
1. ceo_nm : 대표자명
1. crp_nm : 정식명칭
1. crp_nm_e : 영문명칭
1. crp_nm_i : 종목명(상장사) 또는 약식명칭(기타법인)
1. err_code : 에러코드(오류 메시지 참조)
1. err_msg : 에러메시지(오류 메시지 참조)
1. est_dt : 설립일(YYYYMMDD)
1. fax_no : 팩스번호
1. hm_url : 홈페이지
1. ind_cd : 업종코드
1. ir_url : IR홈페이지
1. stock_cd : 상장회사인 경우 주식의 종목코드
1. phn_no : 전화번호

In [2]:
# 1. 기업정보 Json 출력
def response_json(url, crpcode):
    import requests, json
    url      = url.format("8e30368d048361ba865f916c30739160a0e58ddd", crpcode)
    response = requests.get(url)
    return json.loads(response.content)

url      = "http://dart.fss.or.kr/api/company.json?auth={0}&crp_cd={1}"
code     = "035420"
response_json(url, code)

{'acc_mt': '12',
 'adr': '경기도 성남시 분당구  불정로 6',
 'bsn_no': '2208162517',
 'ceo_nm': '한성숙',
 'crp_cls': 'Y',
 'crp_nm': '네이버(주)',
 'crp_nm_e': 'NAVER Corporation',
 'crp_nm_i': 'NAVER',
 'crp_no': '1101111707178',
 'err_code': '000',
 'err_msg': '정상',
 'est_dt': '19990602',
 'fax_no': '031-784-4175',
 'hm_url': 'www.navercorp.com',
 'ind_cd': '63120',
 'ir_url': '',
 'phn_no': '1588-3830',
 'stock_cd': '035420'}

<br>
### **2) 공시정보 수집**
여기에서 auth 값은 개인의 인증키 문자열이다. 이 뒤에 다음과 같은 파라미터를 추가하여야 한다.
1. bsn_tp : 정기공시(5), 주요사항보고(3), 발행공시(11), 지분공시(4), 기타공시(9), 외부감사관련(3), 펀드공시(3), 자산유동화(6), 거래소공시(6), 공정위공시(5) (상세 유형 참조)
1. callback : 콜백함수명(JSONP용)
1. crp_cd : 공시대상회사의 종목코드(상장사:숫자 6자리) 또는 고유번호(기타법인:숫자 8자리)
1. dsp_tp : 정기공시(A), 주요사항보고(B), 발행공시(C), 지분공시(D), 기타공시(E), 외부감사관련(F), 펀드공시(G), 자산유동화(H), 거래소공시(I), 공정위공시(J)
1. end_dt : 검색종료 접수일자(YYYYMMDD) : 없으면 당일
1. fin_rpt : 최종보고서만 검색여부(Y or N) 기본값 : N (정정이 있는 경우 최종정정만 검색)
1. series : 오름차순(asc), 내림차순(desc) 기본값 : desc
1. start_dt : 검색시작 접수일자(YYYYMMDD) : 없으면 end_dt crp_cd가 없는 경우 검색기간은 3개월로 제한
1. sort : 접수일자(date), 회사명(crp), 보고서명(rpt) 기본값 : date
1. page_no : 페이지 번호(1~n) 기본값 : 1
1. page_set : 페이지당 건수(1~100) 기본값 : 10, 최대값 : 100
1. err : 에러코드(오류 메시지 참조)
1. err_msg : 에러메시지(오류 메시지 참조)
1. page_no : 페이지 번호
1. page_set : 페이지당 건수
1. total_count : 총 건수
1. total_page : 총 페이지 수

In [3]:
date       = "20180301"
url_search = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt=" + date
code       = "005930"
data       = response_json(url_search, code)
print('url info >> \nkeys : {}\ntotal page num :{}'.format(data.keys(), data['page_set']))

# 해당일 공개된 공시들 목록
# 1 page 당 8개를 출력
for i in data['list']:
    print(i["rpt_nm"])

url info >> 
keys : dict_keys(['err_code', 'err_msg', 'page_no', 'page_set', 'total_count', 'total_page', 'list'])
total page num :10
기타경영사항(자율공시)
특수관계인과의보험거래
사외이사의선임ㆍ해임또는중도퇴임에관한신고
사외이사의선임ㆍ해임또는중도퇴임에관한신고
사외이사의선임ㆍ해임또는중도퇴임에관한신고
정기주주총회결과
대표이사(대표집행임원)변경(안내공시)
임원ㆍ주요주주특정증권등소유상황보고서
[기재정정]주식분할결정
중간(분기)배당을위한주주명부폐쇄(기준일)결정


In [4]:
# page 에 따른 공시정보 추출 (2page 추출)
page_num   = "1"
url        = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt=" + date + "&page_no=" + page_num
data       = response_json(url, code)
print(len(data['list']))
for i in data['list']:
    print(i["rpt_nm"])

10
기타경영사항(자율공시)
특수관계인과의보험거래
사외이사의선임ㆍ해임또는중도퇴임에관한신고
사외이사의선임ㆍ해임또는중도퇴임에관한신고
사외이사의선임ㆍ해임또는중도퇴임에관한신고
정기주주총회결과
대표이사(대표집행임원)변경(안내공시)
임원ㆍ주요주주특정증권등소유상황보고서
[기재정정]주식분할결정
중간(분기)배당을위한주주명부폐쇄(기준일)결정


In [5]:
# 10개 공시중 1번쨰 정보
data['list'][1]

{'crp_cd': '005930',
 'crp_cls': 'Y',
 'crp_nm': '삼성전자',
 'flr_nm': '삼성전자',
 'rcp_dt': '20180323',
 'rcp_no': '20180323000599',
 'rmk': '공',
 'rpt_nm': '특수관계인과의보험거래'}

1. crp_cd : 공시대상회사의 종목코드(6자리) 또는 고유번호(8자리)
1. crp_cls : 법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
1. crp_nm : 공시대상회사의 종목명(상장사) 또는 법인명(기타법인)
1. flr_nm : 공시 제출인명
1. rcp_dt : 공시 접수일자(YYYYMMDD)
1. rcp_no : 접수번호(공시뷰어 연결에 이용)
1. rmk : 조합된 문자로 각각은 아래와 같은 의미가 있음
    1. 유 : 본 공시사항은 한국거래소 유가증권시장본부 소관임
    1. 코 : 본 공시사항은 한국거래소 코스닥시장본부 소관임
    1. 채 : 본 문서는 한국거래소 채권상장법인 공시사항임
    1. 넥 : 본 문서는 한국거래소 코넥스시장 소관임
    1. 공 : 본 공시사항은 공정거래위원회 소관임
    1. 연 : 본 보고서는 연결부분을 포함한 것임
    1. 정 : 본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람
    1. 철 : 본 보고서는 철회(간주)되었으니 관련 철회신고서(철회간주안내)를 참고하시기 바람
1. rpt_nm : 공시구분+보고서명+기타정보
    1. [기재정정] : 본 보고서명으로 이미 제출된 보고서의 기재내용이 변경되어 제출된 것임
    1. [첨부정정] : 본 보고서명으로 이미 제출된 보고서의 첨부내용이 변경되어 제출된 것임
    1. [첨부추가] : 본 보고서명으로 이미 제출된 보고서의 첨부서류가 추가되어 제출된 것임
    1. [변경등록] : 본 보고서명으로 이미 제출된 보고서의 유동화계획이 변경되어 제출된 것임
    1. [연장결정] : 본 보고서명으로 이미 제출된 보고서의 신탁계약이 연장되어 제출된 것임
    1. [발행조건확정] : 본 보고서명으로 이미 제출된 보고서의 유가증권 발행조건이 확정되어 제출된 것임
    1. [정정명령부과] : 본 보고서에 대하여 금융감독원이 정정명령을 부과한 것임
    1. [정정제출요구] : 본 보고서에 대하여 금융감독원이 정정제출요구을 부과한 것임

In [6]:
data

{'err_code': '000',
 'err_msg': '정상',
 'list': [{'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180323',
   'rcp_no': '20180323801247',
   'rmk': '유',
   'rpt_nm': '기타경영사항(자율공시)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180323',
   'rcp_no': '20180323000599',
   'rmk': '공',
   'rpt_nm': '특수관계인과의보험거래'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180323',
   'rcp_no': '20180323000590',
   'rmk': '',
   'rpt_nm': '사외이사의선임ㆍ해임또는중도퇴임에관한신고'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180323',
   'rcp_no': '20180323000581',
   'rmk': '',
   'rpt_nm': '사외이사의선임ㆍ해임또는중도퇴임에관한신고'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180323',
   'rcp_no': '20180323000572',
   'rmk': '',
   'rpt_nm': '사외이사의선임ㆍ해임또는중도퇴임에관한신고'},
  {'cr

<br>
## **3. Dart 에서 재무제표 Download**
https://datascienceschool.net/view-notebook/adead36729704e7b8660dda3be6a6524/
1. rcp_no : 2번의 Key중 **접수번호(공시뷰어 연결에 이용)를** 활용하여 데이터 수집
    1. http://dart.fss.or.kr/dsaf001/main.do?rcpNo=xxx
1. 해당 페이지의 **dcm 번호**를 알면 **XBRL 및 XLS**를 다운로드 가능

In [7]:
import requests, re, lxml.html

# 공시 Data Info
datum   = data['list'][8]

# 해당 page의 DCM code 추출
url     = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + datum['rcp_no']
req     = requests.get(url)
tree    = lxml.html.fromstring(req.text)
onclick = tree.xpath('//*[@id="north"]/div[2]/ul/li[1]/a')[0].attrib['onclick']
pattern = re.compile("^openPdfDownload\('\d+',\s*'(\d+)'\)")
dcm     = pattern.search(onclick).group(1)
print('rpt_nm   : {}\nrcp_no   : {}\ndcm code : {}'.format(\
       datum['rpt_nm'], datum['rcp_no'],  dcm))

rpt_nm   : [기재정정]주식분할결정
rcp_no   : 20180316800856
dcm code : 6000758


In [8]:
# 엑셀파일 다운로드
url_excel = 'http://dart.fss.or.kr/pdf/download/excel.do?lang=ko&rcp_no={}&dcm_no={}'
url_excel = url_excel.format(datum['rcp_no'], dcm)
url_excel = 'http://dart.fss.or.kr/pdf/download/excel.do?lang=ko&rcp_no=20160516003174&dcm_no=5146351'

pd.read_excel(url_excel,
             sheet_name=u'\uc5f0\uacb0 \ud3ec\uad04\uc190\uc775\uacc4\uc0b0\uc11c',
             skiprows=7, index_col=0, header=[0, 1]).fillna("")

제 48 기 1분기          제 47 기 1분기            \
                                          3개월       누적        3개월        누적   
당기순이익(손실)                             5252803  5252803    4625815   4625815   
기타포괄손익                                -161195  -161195    -819980   -819980   
    후속적으로 당기손익으로 재분류되지 않는 포괄손익         -22420   -22420     -24548    -24548   
        순확정급여부채 재측정요소                  -22783   -22783     -24103    -24103   
        관계기업 및 공동기업의 기타포괄손익에 대한 지분        363      363       -445      -445   
    후속적으로 당기손익으로 재분류되는 포괄손익           -138775  -138775    -795432   -795432   
        매도가능금융자산평가손익                    74595    74595     264851    264851   
        관계기업 및 공동기업의 기타포괄손익에 대한 지분    -111942  -111942     -43607    -43607   
        해외사업장환산외환차이                   -101428  -101428   -1016676  -1016676   
총포괄손익                                 5091608  5091608    3805835   3805835   
포괄손익의 귀속                                                                      
    지배기업 소유주지분                        5070139  5070139    3685109   3685109   
    비지배지분                               21469    21469     120726    120726   

                                               제 47 기             제 46 기  
                                   Unnamed: 5_level_1 Unnamed: 6_level_1  
당기순이익(손실)                                    19060144           23394358  
기타포괄손익                                          76071           -1993414  
    후속적으로 당기손익으로 재분류되지 않는 포괄손익                 288047            -720945  
        순확정급여부채 재측정요소                          263978            -710318  
        관계기업 및 공동기업의 기타포괄손익에 대한 지분              24069             -10627  
    후속적으로 당기손익으로 재분류되는 포괄손익                   -211976           -1272469  
        매도가능금융자산평가손익                          -414961            -232105  
        관계기업 및 공동기업의 기타포괄손익에 대한 지분             -65330            -118305  
        해외사업장환산외환차이                            268315            -922059  
총포괄손익                                        19136215           21400944  
포괄손익의 귀속                                                                  
    지배기업 소유주지분                               18804189           20990732  
    비지배지분                                      332026             410212